In [4]:
# import js2py
import requests
from bs4 import BeautifulSoup as bs

In [5]:

## Function
## EMAIL DECODE
def hex_to_decimal(string, index):
    hex_substring = string[index : index + 2]
    return int(hex_substring, 16)

def decode_email(encoded_string, start_index):
    decoded_string = ""
    xor_value = hex_to_decimal(encoded_string, start_index)

    i = start_index + 2
    while i < len(encoded_string):
        xor_result = hex_to_decimal(encoded_string, i) ^ xor_value
        decoded_string += chr(xor_result)
        i += 2

    try:
        decoded_string = bytes(decoded_string, 'utf-8').decode('unicode_escape')
    except UnicodeDecodeError as error:
        print(f"Error decoding email: {error}")

    return decoded_string

def extract_info(soup):
    contractor_name = soup.find('strong').text.strip()
    address_info = '\n'.join(item.strip() for item in soup.find('td').stripped_strings)

    #### Contact
    contact_data = soup.find('td',{'class':'wide'})
    telephone = contact_data.find('a', class_='phone-link').text.strip()
    # email = soup.find('a', href=lambda x: x and 'email-protection' in x.get('href')).text.strip()

    # Find the span element containing the encoded email
    encoded_email_span = contact_data.find('span', {'data-cfemail': True})

    if encoded_email_span:
        encoded_email = encoded_email_span['data-cfemail']
        start_index = 0  # Replace with your actual start index
        decoded_email = decode_email(encoded_email, start_index)
        email = decoded_email
        # print(decoded_email)
    else:
        email = ""
        print("^"*50)
        print("Encoded Email not found in the HTML.")


    website_element = contact_data.find('a', href=lambda x: x and 'http' in str(x))
    website = website_element.text.strip() if website_element else None

    service_areas = [area.text.strip() for area in soup.find_all('span')[1:]]

    return {
        'contractor_name' : contractor_name,
        'address_info' : address_info,
        'telephone' : telephone,
        'email' : email,
        'website' : website,
        'service_areas' : service_areas
    }

def print_info(data):
    print("Contractor Name:", data['contractor_name'])
    print("\nAddress:")
    print(data['address_info'])
    print("\nContact Information:")
    print("Telephone:", data['telephone'])
    print("Email:", data['email'])
    print("Website:", data['website'])
    print("\nService Area(s):")
    for area in data['service_areas']:
        print(area)



In [6]:
# get_csv
import csv

csv_file_path = r'states.csv'

with open(csv_file_path, 'r') as file:
    # Using csv.reader to read the CSV data
    csv_reader = csv.reader(file, delimiter=',')

    # Skip the header row
    next(csv_reader)

    # Loop through the rows and print the data
    state_data = []
    for row in csv_reader:
        state, region = row
        state_data.append({"state_code":state,"state_name":region})
    print(f"[*] State Readed : {len(state_data)}")

[*] State Readed : 308


In [ ]:
##### MAIN PAGE #####
MAIN = "https://www.powersweeping.org/"
URL = MAIN + "sweep-results/"
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
}

for state_entry in state_data[:5]:
    print("[*] State Selected : ",  state_entry['state_name'])
    Payload = {
        'frmService_area': state_entry['state_code'],
        'searchtype': 'simple',
        'test_text': ''
    }
    response = requests.post(URL, headers =headers, data=Payload)

    if response.status_code == 200:
        try:
            soup = bs(response.content, 'lxml')
            data = soup.find_all('div',{'class':'contractor-result'})
            print(f'Data Found : {len(data)}')
            for entry in data :
                extracted_data = extract_info(entry)

                print_info(data=extracted_data)
                print("*"*50)
        except Exception as e:
            print("An error occurred:", str(e))
    else:
        print('[*] Failed to pass ')
    print('#'*50)
print("[*] Scraper Task Completed")